In [1406]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
os.getcwd()
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [1407]:
import re
train  = pd.read_csv('../input/titanic/train.csv')
test  = pd.read_csv('../input/titanic/test.csv')
y = train['Survived'].values

train.head()
temp = ''
cabs = list(train['Cabin'][:])
cab_let = cabs[:]
cab_num = cabs[:]
for i in range(len(cabs)):
    if type(cabs[i]) == str:
        temp = re.findall('[a-zA-Z]', cabs[i])
        cab_let[i] = temp[0]
        temp = []
        temp = re.findall('[0-9]+', cabs[i])
        if len(temp) != 0:
            cab_num[i] = int(temp[0])
        else:
            cab_num[i] = 0
        temp = []
train['cab_num'] = cab_num
train['cab_let'] = cab_let
train.drop(columns = ['Cabin'])

train['Sex'][train['Sex'] == 'female'] = 1
train['Sex'][train['Sex'] == 'male'] = 0


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [1408]:
train.iloc[10:30]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,cab_num,cab_let
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",1,4.0,1,1,PP 9549,16.7000,G6,S,6.0,G
11,12,1,1,"Bonnell, Miss. Elizabeth",1,58.0,0,0,113783,26.5500,C103,S,103.0,C
12,13,0,3,"Saundercock, Mr. William Henry",0,20.0,0,0,A/5. 2151,8.0500,NaN,S,NaN,NaN
13,14,0,3,"Andersson, Mr. Anders Johan",0,39.0,1,5,347082,31.2750,NaN,S,NaN,NaN
14,15,0,3,"Vestrom, Miss. Hulda Amanda Adolfina",1,14.0,0,0,350406,7.8542,NaN,S,NaN,NaN
15,16,1,2,"Hewlett, Mrs. (Mary D Kingcome)",1,55.0,0,0,248706,16.0000,NaN,S,NaN,NaN
16,17,0,3,"Rice, Master. Eugene",0,2.0,4,1,382652,29.1250,NaN,Q,NaN,NaN
17,18,1,2,"Williams, Mr. Charles Eugene",0,NaN,0,0,244373,13.0000,NaN,S,NaN,NaN
18,19,0,3,"Vander Planke, Mrs. Julius (Emelia Maria Vande...",1,31.0,1,0,345763,18.0000,NaN,S,NaN,NaN
19,20,1,3,"Masselmani, Mrs. Fatima",1,NaN,0,0,2649,7.2250,NaN,C,NaN,NaN


In [1409]:
import sklearn
from sklearn.preprocessing import OneHotEncoder
"""
Dummy-encoding of 'Embarked'
C, S, Q
"""
Snum = sum(sum([train['Embarked'] == 'S']))
Cnum = sum(sum([train['Embarked'] == 'C']))
Qnum = sum(sum([train['Embarked'] == 'Q']))

enc = OneHotEncoder(handle_unknown='ignore')
X = [['S', Snum], ['C', Cnum], ['Q', Qnum]]
enc.fit(X)
dummy_embarked = enc.transform(X).toarray()
for i in range(len(train)):
    if train['Embarked'][i] == 'S':
        train['Embarked'][i] = dummy_embarked[0]
    elif train['Embarked'][i] == 'C':
        train['Embarked'][i] = dummy_embarked[1]
    elif train['Embarked'][i] == 'Q':
        train['Embarked'][i] = dummy_embarked[2]
        
emberked_list = []
for i in range(len(train['Embarked'])):
    emberked_list.append(train['Embarked'][i])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [1410]:
cab_let_uniqs = train['cab_let'].unique()
nan_n = sum(pd.isna(train['cab_let']))
nums = []
for i in range(1, len(cab_let_uniqs)):
    nums.append(sum(train['cab_let'] == cab_let_uniqs[i]))
for i in range(len(train['cab_let'])):
    for j in range(1, len(cab_let_uniqs)):
        if train['cab_let'][i] == cab_let_uniqs[j]:
            train['cab_let'][i] = round(1/nums[j-1], 6)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [1411]:
train['cab_num'][train['cab_num'].isna() == True] = 0
train['cab_let'][train['cab_let'].isna() == True] = 0
train['Age'][train['Age'].isna() == True] = 0
nn = sum(train['Age'] == 0)
train['Age'][train['Age'] == 0] = int(sum(train['Age'])/(len(train) - nn))
train = train.drop(columns = ['Cabin','Name', 'Ticket', 'Embarked', 'PassengerId', 'Survived']) #'Sex', 'Age', 'SibSp', 'Fare'

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doi

In [1412]:
for i in range(len(emberked_list)):
    if type(emberked_list[i]) == float:
        print(i)
emberked_list[61] = [0]*6
emberked_list[829] = [0]*6

61
829


In [1413]:
(train)

,Pclass,Sex,Age,SibSp,Parch,Fare,cab_num,cab_let
0,3,0,22.0,1,0,7.2500,0.0,0
1,1,1,38.0,1,0,71.2833,85.0,0.016949
2,3,1,26.0,0,0,7.9250,0.0,0
3,1,1,35.0,1,0,53.1000,123.0,0.016949
4,3,0,35.0,0,0,8.0500,0.0,0
...,...,...,...,...,...,...,...,...
886,2,0,27.0,0,0,13.0000,0.0,0
887,1,1,19.0,0,0,30.0000,42.0,0.021277
888,3,1,29.0,1,2,23.4500,0.0,0
889,1,0,26.0,0,0,30.0000,148.0,0.016949


In [1414]:
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [1415]:
X = np.array(train)
X = X.tolist()
emberked_list = (np.array(emberked_list)).tolist()
for i in range(len(X)):
    X[i] += emberked_list[i]

In [1416]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

kf = KFold(n_splits = 9)

# reg = LinearRegression().fit(X, y)

In [1417]:
X = np.asarray(X)
np.shape(X)

(891, 14)

In [1418]:

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg = LinearRegression().fit(X_train, y_train)
    y_pred = abs(np.round(reg.predict(X_test)))
    print(accuracy_score(y_test, y_pred, normalize=True, sample_weight=None))

0.8181818181818182
0.8181818181818182
0.7575757575757576
0.8484848484848485
0.7777777777777778
0.7272727272727273
0.7373737373737373
0.8383838383838383
0.8181818181818182


In [1419]:
np.shape(X)

(891, 14)

**TEST** 

In [1420]:
import re
temp = ''
cabs = list(test['Cabin'][:])
cab_let = cabs[:]
cab_num = cabs[:]
for i in range(len(cabs)):
    if type(cabs[i]) == str:
        temp = re.findall('[a-zA-Z]', cabs[i])
        cab_let[i] = temp[0]
        temp = []
        temp = re.findall('[0-9]+', cabs[i])
        if len(temp) != 0:
            cab_num[i] = int(temp[0])
        else:
            cab_num[i] = 0
        temp = []
test['cab_num'] = cab_num
test['cab_let'] = cab_let
test.drop(columns = ['Cabin'])

test['Sex'][test['Sex'] == 'female'] = 1
test['Sex'][test['Sex'] == 'male'] = 0

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [1421]:
import sklearn
from sklearn.preprocessing import OneHotEncoder
"""
Dummy-encoding of 'Embarked'
C, S, Q
"""
Snum = sum(sum([test['Embarked'] == 'S']))
Cnum = sum(sum([test['Embarked'] == 'C']))
Qnum = sum(sum([test['Embarked'] == 'Q']))

# enc = OneHotEncoder(handle_unknown='ignore')
# X = [['S', Snum], ['C', Cnum], ['Q', Qnum]]
# enc.fit(X)
# dummy_embarked = enc.transform(X).toarray()
for i in range(len(test)):
    if test['Embarked'][i] == 'S':
        test['Embarked'][i] = dummy_embarked[0]
    elif test['Embarked'][i] == 'C':
        test['Embarked'][i] = dummy_embarked[1]
    elif test['Embarked'][i] == 'Q':
        test['Embarked'][i] = dummy_embarked[2]
        
        
emberked_list = []
for i in range(len(test['Embarked'])):
    emberked_list.append(test['Embarked'][i])
    
cab_let_uniqs = test['cab_let'].unique()
nan_n = sum(pd.isna(test['cab_let']))    

nums = []
for i in range(1, len(cab_let_uniqs)):
    nums.append(sum(test['cab_let'] == cab_let_uniqs[i]))
for i in range(len(test['cab_let'])):
    for j in range(1, len(cab_let_uniqs)):
        if test['cab_let'][i] == cab_let_uniqs[j]:
            test['cab_let'][i] = round(1/nums[j-1], 6)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be s

In [1422]:
cab_let_uniqs = test['cab_let'].unique()
nan_n = sum(pd.isna(test['cab_let']))
nums = []
for i in range(1, len(cab_let_uniqs)):
    nums.append(sum(test['cab_let'] == cab_let_uniqs[i]))
for i in range(len(test['cab_let'])):
    for j in range(1, len(cab_let_uniqs)):
        if test['cab_let'][i] == cab_let_uniqs[j]:
            test['cab_let'][i] = round(1/nums[j-1], 6)

test['cab_num'][test['cab_num'].isna() == True] = 0
test['cab_let'][test['cab_let'].isna() == True] = 0
test['Age'][test['Age'].isna() == True] = 0
nn = sum(test['Age'] == 0)
test['Age'][test['Age'] == 0] = int(sum(test['Age'])/(len(test) - nn))
test = test.drop(columns = ['Cabin','Name', 'Ticket', 'Embarked', 'PassengerId']) #'Sex', 'Age', 'SibSp', 'Fare'


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/opt/

In [1423]:
test[test['Fare'].isnull()] = 35.5

In [1424]:
# sum(test['Fare'])/len(test)

In [1425]:
for i in range(len(emberked_list)):
    if type(emberked_list[i]) == float:
        print(i)
# emberked_list[61] = [0]*6
# emberked_list[829] = [0]*6
Z = np.array(test)
Z = Z.tolist()
emberked_list = (np.array(emberked_list)).tolist()
for i in range(len(Z)):
    Z[i] += emberked_list[i]
    
Z = np.asarray(Z)

In [1426]:
answer = (abs(np.round(reg.predict(Z))))
answer[152] = 1
set(answer)

{0.0, 1.0}

In [1427]:
os.getcwd()

'/kaggle/working'

In [1429]:
result = pd.read_csv('../input/titanic/gender_submission.csv')
result['Survived'] = [int(x) for x in answer]
result.to_csv('submission_1.csv', index=False)
result.iloc[150:155]

,PassengerId,Survived
150,1042,1
151,1043,0
152,1044,1
153,1045,1
154,1046,0


In [1430]:
np.shape(result)

(418, 2)

In [1431]:
result.iloc[0]

PassengerId    892
Survived         0
Name: 0, dtype: int64